*Load packages and data:*

In [1]:
%run stdPackages.ipynb
read = {'variables': ['Fundamentals', 'Load', 'Generators_Other'], 
        'variable2D': ['Generators_FuelMix','HourlyVariation'],
        'scalars': ['MWP'],
        'maps': ['Generators_Categories']}
db = dbFromWB(os.path.join(d['data'],'E2.xlsx'), read)
db.updateAlias(alias=[('h','h_alias')])
readSets(db)

# Exercise 2: Measures of economic value for competing technologies

The main objectives of the second exercise are:
1. Investigating the relation between hourly variation in demand/supply, installed generation capacity, and profitability of different technologies;
2. explore how this variation relates to policies aimed at reducing emissions;
3. and learn to interact with the code and model classes.

The exercise builds on the ```mBasicInt``` model class; you can find helpful information on this in the following:
* The note "*Models for Energy Economics*" describes the model in section 2.
* The notebook [M_mBasicInt](M_mBasicInt.ipynb) includes some description of the model class.
* The notebook [E2_SolutionGuide](E2_SolutionGuide.ipynb) provides a short solution to the problems posted in the current notebook.

The first parts of the exercise produces tables and figures that are used in section 2.2.

## The setup:

We consider a ```mBasicInt``` model where four different electricity producing plants contribute to satisfy a "daily" load of $400$ GJ. Three of the plant suse a combination of fuels: Coal, natural gas, and bio mass. The last plant produces wind power. The parameter ```FuelMix``` stored in the ```db``` database shows the input intensities:

In [2]:
db['FuelMix'].unstack()

id,id1,id2,id3,id4
BFt,,,,
Coal,1.35,0,0,0
NatGas,0,1.3,0,0
BioMass,0,0,1.2,0


The day consists of four short run states that we refer to as "hours" for convenience. The hourly load peaks at mid-day (states 2,3) and is lowest early morning and night (states 1,4). The wind power plant though, primarily produces energy in the early morning and night; thus, there is a **negative correlation** between load variation and wind generation variation. Consumers have a willingness to pay to avoid load shedding at $25$€ per GJ; this is significantly higher then the most expensive plant. Thus, load shedding will only occur if there is no other way to establish equilibrium. 

Figures 2.1 and 2.2 from the note *Models for Energy Economics* are plotted here for convenience to provide an overview:

In [3]:
IFrame("Misc/Figs/mBasicInt_hvt.pdf", width=600, height=300)

In [4]:
IFrame("Misc/Figs/mBasicInt_MeritOrder.pdf", width=600, height=300)

### Problem 1: Utilization of generation capacities

The four plants have a maximum hourly generation capacity of ```GeneratingCapacity```: For the first three *dispatchable* plants this generation capacity is constant across hours. For the wind turbine (```id4```), generating capacity equals its maximum only in hour $1$. The hourly generation capacity are programmed as the property ```m.hourlyGeneratingCapacity```. The optimal hourly generation is stored as ```Generation``` when the model is solved. We define the daily average theoretical (TCF) and practical (PCF) capacity factors for the utilization of installed capacity from:

$$\begin{align}
    TCF_{id} &= \sum_{h=1}^H \dfrac{\text{Generation}_{id, h}}{q_{id}^{day}}, \qquad q_{id}^{day} \equiv H \cdot \text{GeneratingCapacity}_{id} \\
    PCF_{id} &= \sum_{h=1}^H \dfrac{\text{Generation}_{id, h}}{\tilde{q}_{id}^{day}}, \qquad \tilde{q}_{id}^{day} \equiv \sum_h \text{hourlyGeneratingCapacity}_{id, h} 
\end{align}$$ 

*Solve the model. Compute the theoretical and practical daily average capacity factors for the four plants. Explain briefly the TCF patterns for dispatchable plants and why TCF and PCF differs for the wind turbine.*

In [5]:
m = mBasicInt.mBasicInt(db) # initialize using the database 'db' from above.

In [6]:
m.solve()

Solution status 0: Optimization terminated successfully.


### Problem 2: Average costs of capacity and average costs of energy

In the current model, all plants face two types of costs: 
* Fixed costs (```FOM```): The variable is measured in (1000€ / (GJ/h capacity)) / year. This might look a bit complicated at first, but it is somewhat of an industry standard to report it this way. 
    * First, you should note that "GJ/h capacity" is simply represented as the ```GeneratingCapacity``` variable in our model ($q_i$ in the note).
    * We read FOM as: "It costs 'FOM' $\cdot$ 1000€ every year to maintain 1 unit of capacity $q_i$". 
    * As it is a measure per year, it presumes that there are $H=8760$ hours in the model. In our model with $H=4$ the variable should be scaled appropriately.
* Marginal costs (```mc```): In [exercise 1](E1_Problem.ipynb), we looked into the marginal cost of generation. The variable measures "€/GJ generation".

*Compute the plants' average capacity costs and average energy costs. Compare to their marginal costs and comment briefly.*

### Problem 3: Economic value of capacity

1. *Compute the weighted average marginal system costs across the $h$ hours (use 'HourlyDemand' as weights)*
2. *Define and compute the weighted average marginal system costs for all ids. (use 'Generation')*
3. *Compute the marginal economic value of generating capacity for all plants.*
4. *Comment on the results; relate to results in problems 1-2.*

**Hint 1: Coding help**

The marginal system costs are accessed as

In [ ]:
m.db['λ_equilibrium']

h_alias  _type
1        eq        6.189324
2        eq       15.402685
3        eq       25.000000
4        eq        6.189324
dtype: float64

To remove the ```'_type'``` index level and rename the hour index to 'h', use the shortcut:

In [ ]:
mBasicInt.marginalSystemCosts(m.db)

h
1     6.189324
2    15.402685
3    25.000000
4     6.189324
Name: 0, dtype: float64

**Hint 2: Name of variables in the model**

To compute the marginal economic value of generating capacity, you need to use symbols:

In [17]:
ϑ_upper = - m.db['λ_Generation'].xs('u',level='_type') # this looks in the database 'm.db'
γ = m.hourlyCapFactors # this is a 'property' of the model 'm'
ϑ_upper

h  id 
1  id1    -0.000000
   id2    -0.000000
   id3    -0.000000
   id4     3.189324
2  id1     9.213361
   id2     5.842570
   id3    -0.000000
   id4    12.402685
3  id1    18.810676
   id2    15.439886
   id3     9.597315
   id4    22.000000
4  id1    -0.000000
   id2    -0.000000
   id3    -0.000000
   id4     3.189324
dtype: float64

In [18]:
γ

id   h
id1  1     1.0
     2     1.0
     3     1.0
     4     1.0
id2  1     1.0
     2     1.0
     3     1.0
     4     1.0
id3  1     1.0
     2     1.0
     3     1.0
     4     1.0
id4  1     1.0
     2     0.5
     3    0.25
     4     0.8
dtype: object

### Problem 4: The effect of wind generation and marginal economic value

1. *Simulate an experiment where you gradually change wind generation capacity from 0 to 100.*
2. *Display the effect on (i) The capacity factor (theoretical) and (ii) the marginal economic value of wind.*
3. *Explain.*

**Hint 1:** The capacity factor and marginal economic value of wind are automatically computed and added to the database when the model is solved; you can access them from:

In [19]:
m.db['capacityFactor']

id
id1      0.585
id2        0.5
id3    0.34375
id4     0.6375
dtype: object

In [20]:
m.db['marginalEconomicValue']

id
id1    28.024037
id2    21.282456
id3     9.597315
id4    17.442125
dtype: object

**Hint 2:** To simulate the experiment, use the solution to [Exercise 1](E1_SolutionGuide.ipynb), problem 4 as a guide.